In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import re

1. remove signed keyword

detect occurence of signed keyword 

In [8]:
import os

def find_signed_positions(file_path):
    signed_positions = []
    with open(file_path, 'r') as file:
        # Read the lines of the Verilog file
        lines = file.readlines()
        # Iterate through each line
        for line_num, line in enumerate(lines, start=1):
            # Find all occurrences of "signed" in the line
            occurrences = [match.start() for match in re.finditer(r'\bsigned\b', line)]
            # Add the positions to the list
            signed_positions.extend([(line_num, pos) for pos in occurrences])
    return signed_positions

def main():
    input_folder = "C:/Users/suvan/OneDrive/Desktop/design credit rtl/IntiallyCorrectExamples"

    # Iterate through each file in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.v'):
            input_file_path = os.path.join(input_folder, file_name)

            # Find positions of "signed" in the file
            signed_positions = find_signed_positions(input_file_path)
            if signed_positions:
                print(f"File: {file_name}, Signed Positions: {signed_positions}")

if __name__ == "__main__":
    main()


File: ALU8Bit.v, Signed Positions: [(1, 27), (2, 27), (4, 28)]
File: BothMultipler_4X4.v, Signed Positions: [(5, 13), (6, 8)]
File: WallecTreeMultiplier.v, Signed Positions: [(15, 6), (20, 6), (21, 54)]


Creating 2^n combinations for each file , where n = number of keypoints where error can be added 

In [10]:
import os
UPPER_BOUND = 5000 

def find_signed_positions(verilog_file):
    with open(verilog_file, 'r') as f:
        lines = f.readlines()

    signed_positions = []

    for i, line in enumerate(lines, start=1):
        if "signed" in line:
            signed_positions.append(i)

    return signed_positions

def remove_signed_keyword(verilog_file, output_folder):
    signed_positions = find_signed_positions(verilog_file)

    num_positions = len(signed_positions)
    num_combinations = 2 ** num_positions

    file_name = os.path.basename(verilog_file)
    file_name_base, file_extension = os.path.splitext(file_name)

    with open(verilog_file, 'r') as f_in:
        original_lines = f_in.readlines()

        for i in range(num_combinations):
            new_file_name = f"{file_name_base}_{i}{file_extension}"
            new_file_path = os.path.join(output_folder, new_file_name)

            with open(new_file_path, 'w') as f_out:
                for j, line in enumerate(original_lines, start=1):
                    new_line = line
                    for k in range(num_positions):
                        if i & (1 << k):  # If k-th bit is set in i
                            position = signed_positions[k]
                            if position == j:  # If the position is in the current line
                                new_line = new_line.replace("signed", "")

                    f_out.write(new_line)

def process_all_verilog_files(input_folder, output_folder):
    verilog_files = os.listdir(input_folder)

    for verilog_file in verilog_files:
        if verilog_file.endswith(".v"):
            verilog_file_path = os.path.join(input_folder, verilog_file)
            remove_signed_keyword(verilog_file_path, output_folder)

def main():
    input_folder = "C:/Users/suvan/OneDrive/Desktop/design credit rtl/IntiallyCorrectExamples"
    output_folder = "C:/Users/suvan/OneDrive/Desktop/design credit rtl/Remove signed keyword"

    process_all_verilog_files(input_folder, output_folder)

if __name__ == "__main__":
    main()


In [2]:
UPPER_BOUND = 5000 # for each file it creates min(2^n , UPPER_BOUND) combinations 

def find_signed_positions(verilog_file):
    with open(verilog_file, 'r') as f:
        lines = f.readlines()

    signed_positions = []

    inside_comment_block = False

    for i, line in enumerate(lines, start=1):
        # Check if line starts a comment block
        if '/*' in line:
            inside_comment_block = True
        # Check if line ends a comment block
        if '*/' in line:
            inside_comment_block = False
            continue
        
        # If inside a comment block, skip line
        if inside_comment_block:
            continue

        # Check for signed keyword within the line
        if "signed" in line and not line.strip().startswith("//"):
            signed_positions.append(i)

    return signed_positions

import os

def remove_signed_keyword(verilog_file, output_folder):
    signed_positions = find_signed_positions(verilog_file)

    num_positions = len(signed_positions)
    num_combinations = min(2 ** num_positions , UPPER_BOUND)

    file_name = os.path.basename(verilog_file)
    file_name_base, file_extension = os.path.splitext(file_name)

    with open(verilog_file, 'r') as f_in:
        original_lines = f_in.readlines()

        for i in range(num_combinations):
            new_file_name = f"{file_name_base}_{i}{file_extension}"
            new_file_path = os.path.join(output_folder, new_file_name)

            with open(new_file_path, 'w') as f_out:
                for j, line in enumerate(original_lines, start=1):
                    new_line = line
                    for k in range(num_positions):
                        if i & (1 << k):  # If k-th bit is set in i
                            position = signed_positions[k]
                            if position == j:  # If the position is in the current line
                                new_line = new_line.replace("signed", "")

                    f_out.write(new_line)

def process_all_verilog_files(input_folder, output_folder):
    verilog_files = os.listdir(input_folder)

    for verilog_file in verilog_files:
        if verilog_file.endswith(".v"):
            verilog_file_path = os.path.join(input_folder, verilog_file)
            remove_signed_keyword(verilog_file_path, output_folder)

def main():
    input_folder = "C:/Users/suvan/OneDrive/Desktop/design credit rtl/IntiallyCorrectExamples"
    output_folder = "C:/Users/suvan/OneDrive/Desktop/design credit rtl/Remove signed keyword"

    process_all_verilog_files(input_folder, output_folder)

if __name__ == "__main__":
    main()


2. Negation of variables after assignment 

detect positions of assignment operator 

In [11]:
import os
import re

def find_assignment_positions(verilog_file):
    with open(verilog_file, 'r') as f:
        lines = f.readlines()

    assignment_positions = []

    for i, line in enumerate(lines, start=1):
        matches = re.finditer(r'\b(\w+)\s*=\s*', line)
        for match in matches:
            assignment_positions.append((i, match.start() + 1))  # Adding 1 to match the 1-based indexing of lines

    return assignment_positions

def main():
    input_folder = "C:/Users/suvan/OneDrive/Desktop/design credit rtl/IntiallyCorrectExamples"
    verilog_files = os.listdir(input_folder)

    for verilog_file in verilog_files:
        if verilog_file.endswith(".v"):
            file_path = os.path.join(input_folder, verilog_file)
            assignment_positions = find_assignment_positions(file_path)
            print(f"Assignment positions in {verilog_file}:")
            for position in assignment_positions:
                print(f"Row: {position[0]}, Column: {position[1]}")

if __name__ == "__main__":
    main()


Assignment positions in adder.v:
Assignment positions in AES_128.v:
Row: 72, Column: 12
Row: 73, Column: 12
Row: 74, Column: 12
Row: 75, Column: 12
Row: 83, Column: 12
Row: 84, Column: 12
Row: 85, Column: 12
Row: 86, Column: 12
Row: 91, Column: 12
Row: 134, Column: 12
Row: 135, Column: 12
Row: 136, Column: 12
Row: 137, Column: 12
Row: 167, Column: 12
Row: 168, Column: 12
Row: 169, Column: 12
Row: 170, Column: 12
Assignment positions in ALU16Bit.v:
Row: 9, Column: 12
Row: 10, Column: 12
Row: 11, Column: 12
Row: 16, Column: 12
Row: 18, Column: 12
Row: 20, Column: 12
Row: 22, Column: 12
Row: 24, Column: 12
Row: 26, Column: 12
Row: 28, Column: 12
Row: 30, Column: 12
Row: 32, Column: 12
Row: 34, Column: 12
Row: 36, Column: 12
Row: 38, Column: 12
Row: 40, Column: 12
Row: 42, Column: 12
Row: 44, Column: 12
Row: 46, Column: 13
Row: 46, Column: 27
Row: 47, Column: 20
Assignment positions in ALU8Bit.v:
Row: 14, Column: 7
Row: 15, Column: 7
Row: 16, Column: 7
Row: 18, Column: 25
Row: 19, Column: 

Count number of assignments in each file 

In [1]:
import os
import re

def count_assignment_positions(verilog_file):
    with open(verilog_file, 'r') as f:
        lines = f.readlines()

    num_assignments = 0

    for line in lines:
        matches = re.findall(r'\b\w+\s*=\s*', line)
        num_assignments += len(matches)

    return num_assignments

def main():
    input_folder = "C:/Users/suvan/OneDrive/Desktop/design credit rtl/IntiallyCorrectExamples"
    verilog_files = os.listdir(input_folder)

    for verilog_file in verilog_files:
        if verilog_file.endswith(".v"):
            file_path = os.path.join(input_folder, verilog_file)
            num_assignments = count_assignment_positions(file_path)
            print(f"Number of assignments in {verilog_file}: {num_assignments}")

if __name__ == "__main__":
    main()


Number of assignments in adder.v: 0
Number of assignments in AES_128.v: 17
Number of assignments in ALU16Bit.v: 21
Number of assignments in ALU8Bit.v: 15
Number of assignments in arbiter.v: 0
Number of assignments in ArbitiryCounter.v: 1
Number of assignments in Asynchronous4BitCounter.v: 1
Number of assignments in bar.v: 0
Number of assignments in BiDirectionalCounter.v: 2
Number of assignments in BothMultipler_4X4.v: 8
Number of assignments in c1355.v: 0
Number of assignments in c17.v: 0
Number of assignments in c1908.v: 0
Number of assignments in c2670.v: 90
Number of assignments in c3540.v: 0
Number of assignments in c432.v: 0
Number of assignments in c499.v: 0
Number of assignments in c5315.v: 19
Number of assignments in c6288.v: 0
Number of assignments in c7552.v: 50
Number of assignments in c880.v: 0
Number of assignments in CarrySkipAdder.v: 4
Number of assignments in cavlc.v: 0
Number of assignments in ctrl.v: 1
Number of assignments in dec.v: 0
Number of assignments in Decode

In [7]:
import os
import re
from itertools import combinations

UPPER_BOUND = 5000 # for each file it creates min(2^n , UPPER_BOUND) combinations 

def find_assignment_positions(verilog_file):
    with open(verilog_file, 'r') as f:
        lines = f.readlines()

    assignment_positions = []

    for i, line in enumerate(lines, start=1):
        matches = re.finditer(r'\b(\w+)\s*=\s*', line)
        for match in matches:
            assignment_positions.append((i, match.start() + 1))  # Adding 1 to match the 1-based indexing of lines

    return assignment_positions

def negate_assignments(verilog_file, output_folder):
    assignment_positions = find_assignment_positions(verilog_file)
    num_positions = len(assignment_positions)

    if num_positions > 10:
        assignment_positions = assignment_positions[:10]  # Limiting to 10 positions to avoid excessive combinations

    num_combinations = min(2 ** num_positions, UPPER_BOUND)  # Limiting to UPPER_BOUND combinations

    file_name = os.path.basename(verilog_file)
    file_name_base, file_extension = os.path.splitext(file_name)

    with open(verilog_file, 'r') as f_in:
        original_lines = f_in.readlines()

        for i in range(num_combinations):
            new_file_name = f"{file_name_base}_{i}{file_extension}"
            new_file_path = os.path.join(output_folder, new_file_name)

            with open(new_file_path, 'w') as f_out:
                for j, line in enumerate(original_lines, start=1):
                    new_line = line
                    for k, position in enumerate(assignment_positions):
                        if i & (1 << k):  # If k-th bit is set in i
                            if position[0] == j:  # If the position is in the current line
                                new_line = new_line[:position[1]] + "-" + new_line[position[1]:]

                    f_out.write(new_line)

def process_all_verilog_files(input_folder, output_folder):
    verilog_files = os.listdir(input_folder)

    for verilog_file in verilog_files:
        if verilog_file.endswith(".v"):
            verilog_file_path = os.path.join(input_folder, verilog_file)
            negate_assignments(verilog_file_path, output_folder)

def main():
    input_folder = "C:/Users/suvan/OneDrive/Desktop/design credit rtl/IntiallyCorrectExamples"
    output_folder = "C:/Users/suvan/OneDrive/Desktop/design credit rtl/Negation"

    process_all_verilog_files(input_folder, output_folder)

if __name__ == "__main__":
    main()
